In [ ]:
import random

import numpy as np

from Petri.utils import _return_all_neighboring_locs, _draw, _return_neighboring_unoccupied_locs, _move, _return_neighboring_occupied_locs, _find_connected_components, _sample, _return_random_loc, summarize
from Petri.actions import _scatter, _replicate
from Petri.simulator import simulator
from Petri.elements import Data, Res

In [ ]:
def Dreg(curr_loc, env):
    
    # do something to a neighbor slot
    neighboring_locs = _return_all_neighboring_locs(curr_loc, env)
    neighbor = random.choice(neighboring_locs)

    if neighbor not in env.keys():
        if _draw(.1):  # create res with 30% chance
            env[neighbor] = ("Res", None)
        elif _draw(.01):  # or create a new dreg with 1% chance
            env[neighbor] = ("Dreg", None)
        else:
            pass
    
    # if dreg is next to a occupied slot, randomly destroy neighborhood
    else:
        if _draw(.2):
            neighbors = _return_neighboring_occupied_locs(curr_loc, env)
            for neighbor in neighbors:
                env.pop(neighbor)

    # move to an unoccupied slot (or stay put if there are no unoccupied slots)
    curr_loc, env = _scatter(curr_loc, env)


def Sorter(curr_loc, env):
    # step 0 - replicate
    _replicate(curr_loc, env)
    
    # step 1 - if theres data, move it left or right of the channel
    neighbors = _return_neighboring_occupied_locs(curr_loc, env)
    if neighbors:
        for neighbor in neighbors:
            if env[neighbor][0] == "Data":  # data has id 8
                
                # cache hit, so move data left
                if env[neighbor][1] == env[curr_loc][1]:
                    data_element = env[neighbor]
                    next_loc = (neighbor[0], neighbor[1] - 5)
                    _move(neighbor, next_loc, data_element, env)
                    
                    # cache hit so infect neighboring sorters
                    neighbors = _return_neighboring_occupied_locs(curr_loc, env)
                    if neighbors:
                        for neighbor in neighbors:
                            if env[neighbor][0] == "Sorter":
                                env[neighbor] = env[curr_loc]
                                
                else:
                    data_element = env[neighbor]
                    next_loc = (neighbor[0], neighbor[1] + 5)
                    _move(neighbor, next_loc, data_element, env)
                    
                                
    # step 3 scatter
    _scatter(curr_loc, env)


def Query(curr_loc, env):
    arr = env[curr_loc][1]
    request = np.random.choice(arr, p=[0.6, 0.25, 0.1, 0.05])
    
    # broadcast to sorters
    neighbors = _return_neighboring_occupied_locs(curr_loc, env)
    if neighbors:
        for neighbor in neighbors:
            if env[neighbor][0] == "Sorter":  # sorters is id5
                env[neighbor] = ("Sorter", request)
                
    # scatter
    curr_loc, env = _scatter(curr_loc, env)

In [ ]:
def create_LRU(n_rows, n_cols, n_query, n_dregs, n_sorters, arr):
    world = {}
    world['n_rows'] = n_rows
    world['n_cols'] = n_cols
            
    # create dregs (represented by id 1)
    for _ in range(n_dregs):
        loc = _return_random_loc(n_rows, n_cols)
        world[loc] = ("Dreg", None)
        
    # create sorter (represented by id 5)
    for _ in range(n_sorters):
        loc = _return_random_loc(n_rows, n_cols)
        val = np.random.choice(arr)
        world[loc] = ("Sorter", val)
    
    # create data (represented by id 8)
    for _ in range(30):
        for a in arr:
            loc = _return_random_loc(n_rows, n_cols)
            world[loc] = ("Data", a)
        
    # query is id 10
    for _ in range(n_query): 
        loc = _return_random_loc(n_rows, n_cols)
        world[loc] = ("Query", arr)
        
    return world

In [ ]:
ARR = ["A", "B", "C", "D"]

In [ ]:
loop = {"Query": Query, "Data": Data, "Sorter": Sorter, "Dreg": Dreg, "Res": Res}

n_rows = 20
n_cols = 60
epochs = 100000
ENV = create_LRU(n_rows=n_rows, n_cols=n_cols, n_query=50, n_dregs=5, n_sorters=20, arr=ARR)
CANVASS = np.zeros((n_rows, n_cols))

In [ ]:
simulator(ENV, CANVASS, epochs, loop, frames=100, save=True)

In [ ]:
# we want data elements on the left side of the board

keep = []
for k in ENV.keys():
    if isinstance(k, tuple):
        if k[1] < 30 and ENV[k][0] == "Data":  # data is id8
            keep.append(k)

In [ ]:
counter = {}

for k in keep:
    element = ENV[k]
    if element[1] not in counter.keys():
        counter[element[1]] = 1
    else:
        counter[element[1]] += 1
counter